# An Exploration of TensorFlow

## Introduction

Throughout my exploration of machine learning in the past, I have always been interested in Neural Networks. Neural Networks, in my opinion, appear to be the pinnacle of ML, and I have been excited by their capabilities since the day I first learned about them.

When I heard about TensorFlow, the Neural Network python library from Google, I was ecstatic about its possibilities. Due to this prior interest, I selected an exploration of TensorFlow as my final project for the Big Data class.
While TensorFlow can prove to be difficult to pick up, with a little support from the internet and those around you, it has real potential in implementing neural networks faster than ever before.

For this exploration, I chose a simple problem for the Neural network to solve. Data containing the information of 5 different playing cards would be given as input, and the type of hand would be requested as an output. This problem is one of classification, and in order to most effectively solve it while roaming in a new field, I based my neural networks off of the classification tutorials at www.tensorflow.org. Along with help from Ms. Anderson and my fellow student Malcolm Volk, I created two neural networks to attempt and solve this proposed problem.

### Network 1

A simple Neural Network with zero hidden layers.

I found a dataset from UC: Irvine that contained labeled training and testing data for this problem of card hand classification. This data is used throughout this lab and is cited below.

Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

This network functions without any hidden layers and attempts to classify each hand with the softmax classification model alone.

In [1]:
import tensorflow as tf
import sys
import random
import csv
import numpy as np

#import all data
#import training data
f = open('poker-hand-training-true.data.txt', 'r')
csv_f = csv.reader(f)

training_data, training_label = [],[]

for row in csv_f:
	#create a list for this row's data
	this_data = []
	for i in range(len(row)-1):
		this_data.append(int(row[i]))
	training_data.append(this_data)
	#get this row's label. A list is used in order to keep dimensions the same throughout
	this_label = []
	this_label.append(int(row[len(row)-1]))
	#this_label.append(float(row[len(row)-1])/9)
	#add the label to the full label list
	hot_vector = np.eye(10)[int(row[len(row)-1])]
	training_label.append(hot_vector)

#import testing data

f = open('poker-hand-testing.data.txt', 'r')
csv_f = csv.reader(f)

testing_data, testing_label = [],[]

for row in csv_f:
	#create a list for this row's data
	this_data = []
	for i in range(len(row)-1):
		this_data.append(int(row[i]))
	testing_data.append(this_data)
	#get this row's label. A list is used in order to keep dimensions the same throughout
	this_label = []
	this_label.append(int(row[len(row)-1]))
	
	#add the label to the full label list
	hot_vector = np.eye(10)[int(row[len(row)-1])]
	testing_label.append(hot_vector)

#function to get a batch of training data
def getTrainingBatch(num):
	data = []
	labels = []
	for i in range(num):
		index = int(random.random() * len(training_data))
		data.append(training_data[index])
		labels.append(training_label[index])
	return np.asarray(data),np.asarray(labels)


n_inputs = 10
n_features = 10
n_classes = 10

#initilization of the regression
x = tf.placeholder(tf.float32, [None, n_inputs])
#weights
W = tf.Variable(tf.zeros([n_inputs, n_classes]))
#bias'
b = tf.Variable(tf.zeros([n_classes]))

#implement the softmax regression model 
y = tf.nn.softmax(tf.matmul(x, W) + b)


#training section
#implement our cost function).  This defines how the model should be trained
y_ = tf.placeholder(tf.float32, [None, n_classes])
cost = tf.reduce_mean(tf.abs(tf.sub(y_,y)))

#train the model
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
init = tf.initialize_all_variables()

#launch the model
sess = tf.Session()
sess.run(init)

#train 1000 times
for i in range(1000):
    batch_xs, batch_ys = getTrainingBatch(300)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


#confirm the accuracy of the model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: np.asarray(testing_data), y_: np.asarray(testing_label)}))

#this is testing data for a hand with the identifier 9
print(sess.run(y, feed_dict={x:np.reshape(np.asarray([3,12,3,11,3,13,3,10,3,1]), (1,10))}))

FileNotFoundError: [Errno 2] No such file or directory: 'poker-hand-training-true.data.txt'

Accuracy w/ Softmax: 0.501209

Classify a Royal Flush (Hot vectors):
[[ 0.9332422   0.0380508   0.00398424  0.00368423  0.00352931  0.00351424
   0.00350735  0.00349597  0.0034956   0.00349621]]

After experimenting with various training algorithms and between sigmoid and softmax models, I found that the configuration above of Gradient Descent Optimization and Softmax regression gave the most accurate results.  While these were the most accurate, at around 50% accuracy, this model does not function extremely well.

One hypothesis I had for this phenomenon was the distribution of training data.  Playing card hands are not evenly distributed, and therefore neither is the training data.  Without hidden layers to normalize the network, as seen by testing this dataset with a test row, the predictions are distributed towards the lower hands, identified by 0 and 1.  Since these are very prominent within the dataset, the accuracy of 50% could be from only these hands being classified correctly.  Data that supports this is the attempted classification of a Royal Flush, identified by a 9, which our networks classifies as a 0, with a high probability of a 1.

By adding hidden layers we can attempt to improve accuracy.

## Network 2

This is an improved version of the network created above.  This network was created with the guidance of Malcolm Volk and the Deep MINST for Experts tutorial found here.
https://www.tensorflow.org/versions/r0.9/tutorials/mnist/pros/index.html

This network includes 4 hidden layers two of which use ReLU, a common activation layer function, and two of which use sigmoid regression.  Sigmoid regression was chosen for this problem because it can returns values between 0 and 1.  Our dataset returns 10 possible classes, we can use sigmoid to return a decimal and later convert it to the class.

In [ ]:
import tensorflow as tf
import sys
import random
import csv
import numpy as np

#import all data
#import training data
f = open('poker-hand-training-true.data.txt', 'r')
csv_f = csv.reader(f)

training_data, training_label = [],[]

for row in csv_f:
	#create a list for this row's data
	this_data = []
	for i in range(len(row)-1):
		this_data.append(int(row[i]))
	training_data.append(this_data)
	#get this row's label. A list is used in order to keep dimensions the same throughout
	this_label = []
	this_label.append(float(row[len(row)-1])/9)
	#add the label to the full label list
	#hot_vector = np.eye(10)[int(row[len(row)-1])]
	training_label.append(this_label)

#import testing data

f = open('poker-hand-testing.data.txt', 'r')
csv_f = csv.reader(f)

testing_data, testing_label = [],[]

for row in csv_f:
	#create a list for this row's data
	this_data = []
	for i in range(len(row)-1):
		this_data.append(int(row[i]))
	testing_data.append(this_data)
	#get this row's label. A list is used in order to keep dimensions the same throughout
	this_label = []
	this_label.append(float(row[len(row)-1])/9)
	
	#add the label to the full label list
	testing_label.append(this_label)

#function to get a batch of training data
def getTrainingBatch(num):
	data = []
	labels = []
	for i in range(num):
		index = int(random.random() * len(training_data))
		data.append(training_data[index])
		labels.append(training_label[index])
	return np.asarray(data),np.asarray(labels)

#these functions create more effective weights + bias'
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

#ensure that there will be no 'dead' nuerons by starting with a slightly positive initial bias
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


n_features=10
n_features_2=10
n_features_3=10
n_features_4=10
n_input=10
n_classes=1
n_layers=5

#inputs
x = tf.placeholder(tf.float32, [None, n_input])
y_ = tf.placeholder(tf.float32, [None, n_classes])

#weights
weight_1 = weight_variable([n_input, n_features])
weight_2 = weight_variable([n_features, n_features_2])
weight_3 = weight_variable([n_features_2, n_features_3])
weight_4 = weight_variable([n_features_3, n_features_4])
weight_5 = weight_variable([n_features_4, n_classes])

#bias
bias_1 = bias_variable([n_features])
bias_2 = bias_variable([n_features_2])
bias_3 = bias_variable([n_features_3])
bias_4 = bias_variable([n_features_4])
bias_5 = bias_variable([n_classes])

#model
def model(x):
    hidden_1 = tf.nn.relu(tf.matmul(x, weight_1) + bias_1)
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weight_2) + bias_2)
    hidden_3 = tf.nn.sigmoid(tf.matmul(hidden_2, weight_3) + bias_3)
    hidden_4 = tf.nn.sigmoid(tf.matmul(hidden_3, weight_4) + bias_4)
    output = 10 * tf.nn.sigmoid(tf.matmul(hidden_4, weight_5) + bias_5)
    return output

cost = tf.reduce_mean(tf.abs(tf.sub(model(x),y_)))
train_step = tf.train.GradientDescentOptimizer(.001).minimize(cost)
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

#train 1000 times
for i in range(1000):
    batch_xs, batch_ys = getTrainingBatch(300)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})


accuracy = 1-(tf.reduce_mean(tf.abs(tf.sub(tf.round(model(x)),y_))))/10
print(sess.run(accuracy, feed_dict={x: np.asarray(testing_data), y_: testing_label}))

Accuracy: 0.99318

With this new model we get a much higher accuracy of 99.318%.  This is a huge improvement over the ~50% given by the original network, all from adding a few hidden layers.  This improvement happens so quickly due to the nature of the dataset.  In the original network, there were no layers where the network had time to preform calculations and predict higher value hands.  It was essentially picking by chance due to the fact that most hands were rated either a 0 or a 1.  With multiple hidden layers, deep connections can be made between different cards in a hand and higher accuracy values were achieved.

The functions and configuration chosen for this network were found through trial and error, but I found that many had little impact if any at all.  Most changes would result in less than a tenth of a percent change, but in the end I decided this configuration fit my needs and provided adequate accuracy.  While this many hidden layers may be slightly overkill for this problem, this is an exploration and experimenting with the effects of multiple layers helped me better understand TensorFlow as a whole.

# Conclusion

Throughout my exploration of TensorFlow, I came to a few realizations.  The first realization I had was a confirmation in my interest in neural networks and machine learning overall.  While it was frustrating trying to optimize my original network and still get accuracies of less than ~50%, running a multilayer network and as if magic returning almost perfect accuracy re-enforced my love for this field.  The idea that these systems can learn abstract ideas through data and analytics is fascinating to me, and I will continue pursuing neural networks in the future.

I also found that TensorFlow, as a software, has its pros and cons.  TensorFlow provides a great platform for analyzing data in a variety of forms, and definitely brings neural networks closer in reach to elementary data scientists.  However, there is a quite a steep learning curve.  While one does not have to completely create a network from scratch, it is important to at least partially understand the concepts at hand in order to effectively implement an accurate network.  I found myself very lost at times trying to choose which model worked best, and I had troubles when receiving various errors regarding matrix shape and placeholder tensors.  It took quite a while to diagnose some of these issues, even with the help of my peers working on the same systems.  TensorFlow definitely has the potential to bring neural networks to the masses, but at this point, it is a tool for experienced programmers and those with background in machine learning, and it is not a library that can be picked up quickly.

While I did learn a lot of the course of this project, my understanding of neural networks is far from complete.  When choosing models for hidden and activation layers, I was essentially in the dark.  I still have little comprehension of the differences between various models, and would often try multiple before choosing a method.  Sigmoid regression is one model I do understand, and did use during this project, however others such as ReLU, used as an activation layer, I have little understanding of how it functions at all.  In the future, I would love to understand every line of these networks at a conceptual level, however at the time being, my abstract understanding has served me well.  Due to this exploration, I can more confidently explain the inner workings of a neural network, despite not building one myself.

This conclusion, is the essence of TensorFlow.  While the user may not understand fully what is going on behind the scenes, with a little effort, it is possible to explore and experiment with neural networks.  Though it could be simplified further, TensorFlow does serve it's purpose of facilitating experiments such as this one; without it, I don't think I could have done an analysis of this kind in a time period so short.  TensorFlow while still in it's infancy, is a tool that could evolve into something great.  I am confident I will continue to use it in the future, and I am excited to see how it improves.